### Exotic Options

Exotic options are not traded on exchanges but rather on the OTC (over the counter) market.

#### <span style="color:#22EEFF">European Up-and-out Call Option</span>

<!-- The goal of Submission 1 is to price a European up-and-out call option held with a risky counterparty. -->

The payoff of this option at maturity T is as follows:

$C_{uo}(S_T) = (S_T - K)^+\;\mathbb{1}_{\{S_t < L\}}$ 

where $K$ is the strike of the option, $L$ is the barrier level, and $S_t$ is the share price at time $t$.


In [2]:
import numpy as np
import scipy.stats as stats

def d1(S,K,r,sigma,tau):
    return (np.log(S/K)+(r+sigma**2/2)*(tau))/(sigma*np.sqrt(tau)) 
def d2(S,K,r,sigma,tau):
    return d1(S,K,r,sigma,tau) - sigma*np.sqrt(tau)

def discounted_value(S,K,r,T):
    return np.exp(-r*T)*np.maximum((S-K),0)

T = 1; S = 100; K = S; L = 150; r = 0.08; sigma = 0.3; sigma2 = 0.25; rho = 0.2; t = 0; debt = 175; recovery = 0.25; V = 200;

There a closed-form, analytical solution for pricing an up-and-out barrier call option, as can be found in (Hull 2018).

$C_{uo} = S_0 N(d_1) - Ke^{-rT}N(d_2) - S_0 N(x_1) + Ke^{(-rT} N(x_1 - \sigma\sqrt{T}) +$

$+S_0(L/S_0)^{2\lambda}(N(-y) - N(-y1))- Ke^{-rT}(L/S_0)^{2\lambda-2}(N(-y+\sigma\sqrt{T}) -N(-y1+ \sigma\sqrt{T}))$

where

$d_1 = \dfrac{\ln(S_0/K) + (r-\sigma^2/2)T}{\sigma^2\sqrt{T}} \qquad d_2 = d_1 - \sigma\sqrt{T}$

$\lambda = \dfrac{r+\sigma^2/2}{\sigma^2} \qquad y = \dfrac{\ln(L^2/S_0K)}{\sigma\sqrt{T}}+\lambda\sigma\sqrt{T}$

$x_1 = \dfrac{\ln(S_0/L)}{\sigma\sqrt{T}}+\lambda\sigma\sqrt{T} \qquad y_1 = \dfrac{\ln(L/S_0)}{\sigma\sqrt(T)}+\lambda\sigma\sqrt{T}$

In [3]:
def analytical_value_Call_Up_Out(S0,K,L,r,sigma,T):
    l = (r+sigma**2/2)/(sigma**2)
    x1 = np.log(S0/L)/(sigma*np.sqrt(T)) + l*sigma*np.sqrt(T)
    y1 = np.log(L/S0)/(sigma*np.sqrt(T)) + l*sigma*np.sqrt(T)
    y = np.log(L**2/(S0*K))/(sigma*np.sqrt(T)) + l*sigma*np.sqrt(T)
    lmbd = l; Hb=L

    return  S*stats.norm.cdf(d1(S,K,r,sigma,T)) - K*np.exp(-r*T)*stats.norm.cdf(d2(S,K,r,sigma,T)) - S0*stats.norm.cdf(x1) + \
    K*np.exp(-r*T)*stats.norm.cdf(x1 - sigma*np.sqrt(T)) + S0*(L/S0)**(2*l)*(stats.norm.cdf(-y) \
    - stats.norm.cdf(-y1)) - K*np.exp(-r*T)*(L/S0)**(2*l-2)*(stats.norm.cdf(-y+sigma*np.sqrt(T)) -stats.norm.cdf(-y1+ sigma*np.sqrt(T)))

In [4]:
np.random.seed(42)
dt = 1/10000; upOutCall = [None]*30; sig = [None]*30; c=0
for i in range(1,31):
    W = np.random.randn(i*1000,int(1/dt))*sigma*np.sqrt(dt)
    path = np.cumprod(np.exp((r-(sigma**2)/2.0)*dt+W),1)*S
    path = np.hstack((S*np.ones((i*1000,1)),path))
    ST = path[:,-1]
    ST[np.max(path,axis=1)>L] = 0
    upOutCall_val = discounted_value(ST,K,r,T)
    upOutCall[c] = upOutCall_val.mean()
    sig[c] = upOutCall_val.std()/np.sqrt(i*1000)
    c+=1
analyValue = analytical_value_Call_Up_Out(S,K,L,r,sigma,T)

In [5]:
print('Monte Carlo (dt = 1/10000): %.2f' % np.mean(upOutCall))
print('Analytical (dt = 1/10000): %.3f' % analyValue)
print('Error: %.2f %%' % (100*(np.abs(np.mean(upOutCall)-analyValue)/analyValue)))

Monte Carlo (dt = 1/10000): 5.35
Analytical (dt = 1/10000): 5.313
Error: 0.66 %


In [7]:
C = S*stats.norm.cdf(d1(S,K,r,sigma,T)) - K*np.exp(-r*T)*stats.norm.cdf(d2(S,K,r,sigma,T))

#### <span style="color:#22EEFF">Credit Valuation Adjustment (CVA)</span>

In [12]:
from scipy.stats import norm
np.random.seed(42)
H = np.linalg.cholesky(np.array([[1,rho],[rho,1]]))

amount_lost = [None]*2; cva_est = [None]*2
cva_std = [None]*2; DFValue = [None]*2

m = 1000

dt = 1/m
paths = np.zeros((50000,m+1))
paths[:,0] = S
firm_path = np.zeros((50000,m+1))
firm_path[:,0] = V

for j in range(0,m):
    norm_matrix = np.random.normal(size=(2,50000))
    corr_norm_matrix = np.matmul(H,norm_matrix)
    paths[:,j+1] = paths[:,j]*np.exp((r-sigma**2/2)*dt + sigma*np.sqrt(dt)*corr_norm_matrix[0,:])
    firm_path[:,j+1] = firm_path[:,j]*np.exp((r-sigma2**2/2)*dt + sigma2*np.sqrt(dt)*corr_norm_matrix[1,:])

final_values = paths[:,-1] - K
max_values = np.max(paths,axis=1)
final_values[final_values < 0] = 0
final_values[max_values >= L] = 0

term_firm_val = firm_path[:,-1]
amount_lost = np.exp(-r*T)*(1-recovery)*(term_firm_val < debt)*final_values
cva_est = np.mean(amount_lost)
cva_std = np.std(amount_lost)/np.sqrt(50000)
DFValue = np.mean(final_values*np.exp(-r*T))
    
default_prob = stats.norm.cdf(-d2(V,debt,r,sigma,T))
uncorr_cva = (1-recovery)*default_prob*C
print("Default Free Option Value: %.2f" % (DFValue))
print("Credit Valuation Adjustment: %.2f" % (cva_est))
#print('\nUncorrelated: %.3f' % uncorr_cva)

Default Free Option Value: 5.44
Credit Valuation Adjustment: 0.76
